In [0]:
%sql
SELECT * FROM workspace.sc_silver.deals

In [0]:
%sql
SELECT * FROM workspace.sc_silver.leads_pbs

In [0]:
%sql
SELECT * FROM workspace.sc_silver.campaigns

In [0]:
%sql
SELECT * FROM workspace.sc_silver.contactos_pbs

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.campaigns AS
SELECT
  _fivetran_synced,
  _fivetran_deleted,
  _fivetran_index,
  layout,
  codigo_campanha,
  modified_time,
  created_time,
  campaign_owner_name,
  campaign_name,
  id,
  _fivetran_id
FROM workspace.sc_silver.campaigns;


In [0]:
%sql
SELECT COUNT(*) AS num_duplicate_ids
FROM (
  SELECT id
  FROM workspace.sc_silver.contactos_pbs
  GROUP BY id
  HAVING COUNT(*) > 1
);


In [0]:
%sql
SELECT COUNT(*) AS num_duplicate_converted_contacts
FROM (
  SELECT converted_contact
  FROM workspace.sc_silver.leads_pbs
  WHERE converted_contact IS NOT NULL
  GROUP BY converted_contact
  HAVING COUNT(*) > 1
);


In [0]:
%sql
SELECT COUNT(*) AS total_rows
FROM workspace.sc_silver.leads_pbs;


In [0]:
%sql
SELECT COUNT(id) AS non_null_ids
FROM workspace.sc_silver.contactos_pbs;


In [0]:
%sql
SELECT COUNT(DISTINCT c.id) AS common_unique_ids
FROM workspace.sc_silver.contactos_pbs c
JOIN workspace.sc_silver.leads_pbs l
  ON c.id = l.converted_contact;


In [0]:
%sql
SELECT COUNT(DISTINCT leads_pbs.converted_contact) AS common_ids_between_leads_and_campaigns
FROM workspace.sc_silver.leads_pbs
JOIN workspace.sc_silver.campaigns
  ON leads_pbs.converted_contact = campaigns.id
WHERE leads_pbs.converted_contact IS NOT NULL;


In [0]:
%sql
-- Check which campaign IDs exist in leads.id
SELECT DISTINCT id AS campaign_id_in_leads_id
FROM workspace.sc_silver.campaigns
WHERE id IN (
  SELECT id FROM workspace.sc_silver.leads_pbs
);


In [0]:
%sql
SELECT
  campaigns.id AS campaign_id,
  CASE WHEN leads_by_id.id IS NOT NULL THEN 'Yes' ELSE 'No' END AS found_in_leads_id,
  CASE WHEN leads_by_owner.lead_owner IS NOT NULL THEN 'Yes' ELSE 'No' END AS found_in_lead_owner
FROM workspace.sc_silver.campaigns

LEFT JOIN workspace.sc_silver.leads_pbs AS leads_by_id
  ON campaigns.id = leads_by_id.id

LEFT JOIN workspace.sc_silver.leads_pbs AS leads_by_owner
  ON campaigns.id = leads_by_owner.lead_owner;


In [0]:
%sql
SELECT
  campaigns.id AS campaign_id,
  
  -- Lookup flags
  CASE WHEN leads_by_id.id IS NOT NULL THEN 'Yes' ELSE 'No' END AS found_in_leads_id,
  CASE WHEN leads_by_owner.lead_owner IS NOT NULL THEN 'Yes' ELSE 'No' END AS found_in_lead_owner,

  -- Add columns from leads_pbs
  leads_by_id.id AS id_lead,
  leads_by_id.converted_contact

FROM workspace.sc_silver.campaigns

-- Lookup campaigns.id in leads_pbs.id
LEFT JOIN workspace.sc_silver.leads_pbs AS leads_by_id
  ON campaigns.id = leads_by_id.id

-- Lookup campaigns.id in leads_pbs.lead_owner
LEFT JOIN workspace.sc_silver.leads_pbs AS leads_by_owner
  ON campaigns.id = leads_by_owner.lead_owner;


In [0]:
%sql
SELECT
  c.id AS contacto_id,

  -- Flags de correspondência
  CASE WHEN p1.id IS NOT NULL THEN 'Yes' ELSE 'No' END AS match_with_proposta_id,
  CASE WHEN p2.proposta_realizada_owner IS NOT NULL THEN 'Yes' ELSE 'No' END AS match_with_proposta_owner,
  CASE WHEN p3.id_contacto IS NOT NULL THEN 'Yes' ELSE 'No' END AS match_with_id_contacto

FROM workspace.sc_silver.contactos_pbs c

-- Join com propostas_realizadas.id
LEFT JOIN workspace.sc_silver.propostas_realizadas p1
  ON c.id = p1.id

-- Join com propostas_realizadas.proposta_realizada_owner
LEFT JOIN workspace.sc_silver.propostas_realizadas p2
  ON c.id = p2.proposta_realizada_owner

-- Join com propostas_realizadas.id_contacto
LEFT JOIN workspace.sc_silver.propostas_realizadas p3
  ON c.id = p3.id_contacto;


In [0]:
%sql
SELECT COUNT(DISTINCT p3.id_contacto) AS matching_contact_ids
FROM workspace.sc_silver.propostas_realizadas p3
JOIN workspace.sc_silver.contactos_pbs c
    ON p3.id_contacto = c.id
WHERE p3.id_contacto IS NOT NULL;


In [0]:
%sql
SELECT COUNT(DISTINCT p.id_contacto) AS matching_contact_ids
FROM workspace.sc_silver.propostas_realizadas p
JOIN workspace.sc_silver.deals d
  ON p.id_contacto = d.id_contacto
WHERE p.id_contacto IS NOT NULL;


In [0]:
%sql
DELETE FROM workspace.sc_silver.leads_pbs
WHERE converted_contact IS NULL;


In [0]:
%sql
SELECT COUNT(*) AS total_rows
FROM workspace.sc_silver.leads_pbs;


In [0]:
%sql
SELECT
  l.converted_contact,
  CASE WHEN d1.id IS NOT NULL THEN 'Yes' ELSE 'No' END AS match_in_deals_id,
  CASE WHEN d2.converted_from_lead IS NOT NULL THEN 'Yes' ELSE 'No' END AS match_in_deals_converted_from_lead
FROM workspace.sc_silver.leads_pbs l
LEFT JOIN workspace.sc_silver.deals d1
  ON l.converted_contact = d1.id
LEFT JOIN workspace.sc_silver.deals d2
  ON l.converted_contact = d2.converted_from_lead
WHERE l.converted_contact IS NOT NULL;


In [0]:
%sql
-- How many leads match deals.id
SELECT COUNT(DISTINCT l.id) AS leads_in_deals_id
FROM workspace.sc_silver.leads_pbs l
JOIN workspace.sc_silver.deals d ON l.id = d.id;

-- How many leads match deals.converted_from_lead
SELECT COUNT(DISTINCT l.id) AS leads_in_converted_from_lead
FROM workspace.sc_silver.leads_pbs l
JOIN workspace.sc_silver.deals d ON l.id = d.converted_from_lead;


In [0]:
%sql
SELECT COUNT(DISTINCT d.campanha) AS matched_campaigns_in_deals
FROM workspace.sc_silver.deals d
JOIN workspace.sc_silver.campaigns c
  ON d.campanha = c.id
WHERE d.campanha IS NOT NULL;


In [0]:
%sql
SELECT COUNT(*) AS num_duplicate_converted_contacts
FROM (
  SELECT converted_contact
  FROM workspace.sc_silver.leads_pbs
  WHERE converted_contact IS NOT NULL
  GROUP BY converted_contact
  HAVING COUNT(*) > 1
);


In [0]:
%sql
SELECT
  occurrences,
  COUNT(*) AS num_contacts_with_this_occurrence
FROM (
  SELECT converted_contact, COUNT(*) AS occurrences
  FROM workspace.sc_silver.leads_pbs
  WHERE converted_contact IS NOT NULL
  GROUP BY converted_contact
) AS sub
GROUP BY occurrences
ORDER BY occurrences;


In [0]:
%sql
SELECT COUNT(*) AS num_duplicate_ids
FROM (
  SELECT id
  FROM workspace.sc_silver.campaigns
  GROUP BY id
  HAVING COUNT(*) > 1
);


In [0]:
%sql
SELECT 
  id_contacto, 
  COUNT(*) AS occurrences
FROM workspace.sc_silver.deals
WHERE id_contacto IS NOT NULL
GROUP BY id_contacto
HAVING COUNT(*) > 1
ORDER BY occurrences DESC;


In [0]:
%sql
SELECT
  COUNT(*) AS total_rows,
  COUNT(CASE WHEN apoio_concessionario IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_apoio_concessionario,
  COUNT(CASE WHEN apoio_total IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_apoio_total,
  COUNT(CASE WHEN sub_total_com_extras IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_sub_total_com_extras,
  COUNT(CASE WHEN descricao_do_pedido_de_apoio IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_descricao_do_pedido_de_apoio,
  COUNT(CASE WHEN estado_do_pedido IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_estado_do_pedido,
  COUNT(CASE WHEN apoio_percentual IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_apoio_percentual,
  COUNT(CASE WHEN data_de_criacao_da_proposta IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_de_criacao_da_proposta,
  COUNT(CASE WHEN valor_campanhas_comerciais IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_valor_campanhas_comerciais,
  COUNT(CASE WHEN ofertas_de_campanha IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_ofertas_de_campanha,
  COUNT(CASE WHEN resposta_do_importador IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_resposta_do_importador,
  COUNT(CASE WHEN data_de_entrega_da_proposta IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_de_entrega_da_proposta,
  COUNT(CASE WHEN desconto_total__c__apoio_de_importador_ IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_desconto_total__c__apoio_de_importador_,
  COUNT(CASE WHEN codigo_cor_exterior IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_codigo_cor_exterior,
  COUNT(CASE WHEN codigo_cor_interior IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_codigo_cor_interior,
  COUNT(CASE WHEN concessionario_owner IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_concessionario_owner,
  COUNT(CASE WHEN valor_aprovado IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_valor_aprovado,
  COUNT(CASE WHEN estado_do_contrato IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_estado_do_contrato,
  COUNT(CASE WHEN data_prevista_matricula IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_prevista_matricula,
  COUNT(CASE WHEN data_prevista_de_entrega IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_prevista_de_entrega,
  COUNT(CASE WHEN data_da_conclusao IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_da_conclusao,
  COUNT(CASE WHEN id_classe IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_id_classe,
  COUNT(CASE WHEN descricao_classe IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_descricao_classe,
  COUNT(CASE WHEN id_model_group IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_id_model_group,
  COUNT(CASE WHEN descricao_model_group IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_descricao_model_group,
  COUNT(CASE WHEN valid_until IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_valid_until,
  COUNT(CASE WHEN forma_de_pagamento IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_forma_de_pagamento,
  COUNT(CASE WHEN nome_da_campanha IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_nome_da_campanha,
  COUNT(CASE WHEN _fivetran_index IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_fivetran_index
FROM workspace.sc_silver.propostas_realizadas;


In [0]:
%sql
ALTER TABLE workspace.sc_silver.propostas_realizadas
DROP COLUMNS (
  descricao_do_pedido_de_apoio,
  apoio_percentual,
  valor_campanhas_comerciais,
  ofertas_de_campanha,
  resposta_do_importador,
  valid_until,
  nome_da_campanha
);


In [0]:
%sql
ALTER TABLE workspace.sc_silver.contactos_pbs
DROP COLUMNS (
  last_name,
  row_names,
  email,
  mobile,
  first_name,
  phone,
  date_of_birth,
  email_opt_out,
  salutation,
  tag,
  codigo_postal,
  login_myhyundai,
  full_name,
  pioneiros_myhyundai,
  lead_source,
  nif,
  modo_de_contacto_preferencial,
  distrito,
  pais,
  sexo,
  localidade,
  morada,
  concelho,
  consentimento,
  data_de_recolha_de_consentimento_hyundai,
  description,
  link_centro_consentimento,
  ccupdated
);


In [0]:
%sql
ALTER TABLE workspace.sc_silver.leads_pbs
DROP COLUMNS (
  last_name,
  row_names,
  email,
  mobile,
  telefone_empresa,
  e_mail_empresa,
  first_name,
  zip_code,
  description,
  salutation,
  motivo,
  hmod___hcreat,
  campaign_name,
  campaign_medium,
  campaign_term,
  campaign_content,
  campaignsource,
  con,
  caracterizacao,
  test_check1,
  test_check,
  link_centro_consentimento,
  industry,
  login_myhyundai,
  website,
  full_name,
  pontuacao,
  data_de_recolha_de_consentimento_hyundai,
  ccupdated
);


In [0]:
%sql
ALTER TABLE workspace.sc_silver.deals
DROP COLUMNS (
  row_names,
  tag,
  amount,
  expected_revenue,
  pioneiros_myhyundai,
  lead_source,
  login_myhyundai,
  aprovacao_do_pedido_de_apoio,
  `type`        -- quote reserved word
);


In [0]:
%sql
ALTER TABLE workspace.sc_silver.propostas_realizadas
DROP COLUMNS (
  sales_status,
  row_names,
  tax,
  discount,
  adjustments,
   estado_do_contrato
  );



In [0]:
from pyspark.sql import functions as F

# ==== DEALS ====
df_deals = spark.table("workspace.sc_silver.deals")
df_deals_renamed = df_deals.select(
    *[F.col(c).alias(f"{c}_deals") for c in df_deals.columns]
)
df_deals_renamed.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.deals_renamed")
display(spark.table("workspace.sc_silver.deals_renamed"))

# ==== CAMPAIGNS ====
df_campaigns = spark.table("workspace.sc_silver.campaigns")
df_campaigns_renamed = df_campaigns.select(
    *[F.col(c).alias(f"{c}_campaigns") for c in df_campaigns.columns]
)
df_campaigns_renamed.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.campaigns_renamed")
display(spark.table("workspace.sc_silver.campaigns_renamed"))


In [0]:
from pyspark.sql import functions as F

# Load renamed tables
df_deals_renamed = spark.table("workspace.sc_silver.deals_renamed")
df_campaigns_renamed = spark.table("workspace.sc_silver.campaigns_renamed")

# Join on campanha_deals = id_campaigns
df_deals_with_campaigns = df_deals_renamed.join(
    df_campaigns_renamed,
    df_deals_renamed.campanha_deals == df_campaigns_renamed.id_campaigns,
    "left"
)

# Save as permanent table
df_deals_with_campaigns.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.deals_with_campaigns")

# Preview
display(spark.table("workspace.sc_silver.deals_with_campaigns"))


In [0]:
from pyspark.sql import functions as F

# Load original table
df = spark.table("workspace.sc_silver.propostas_realizadas")

# Build SELECT list with aliases (append _propostas_realizadas to each column)
select_expr = [F.col(c).alias(f"{c}_propostas_realizadas") for c in df.columns]
df_renamed = df.select(*select_expr)

# Write to a NEW table so you don't overwrite original
df_renamed.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.propostas_realizadas_renamed")

# Display the new table
display(spark.table("workspace.sc_silver.propostas_realizadas_renamed"))

In [0]:
#  JOIN propostas_realizadas_renamed with deals_with_campaigns ====

# Load both tables
df_propostas_renamed = spark.table("workspace.sc_silver.propostas_realizadas_renamed")
df_deals_with_campaigns = spark.table("workspace.sc_silver.deals_with_campaigns")

# LEFT join from propostas to deals
df_joined = df_propostas_renamed.join(
    df_deals_with_campaigns,
    df_propostas_renamed.id_contacto_propostas_realizadas == df_deals_with_campaigns.id_contacto_deals,
    "left"
)

# Save as propostas_with_deals_with_campaigns
df_joined.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.propostas_with_deals_with_campaigns")

# Display the result
display(spark.table("workspace.sc_silver.propostas_with_deals_with_campaigns"))

In [0]:
%sql
SELECT *
FROM workspace.sc_silver.leads_pbs
LIMIT 100;


In [0]:
%sql
SELECT *
FROM workspace.sc_silver.contactos_pbs
LIMIT 100;


In [0]:
%sql
SELECT COUNT(DISTINCT c.id) AS common_unique_ids
FROM workspace.sc_silver.contactos_pbs c
JOIN workspace.sc_silver.leads_pbs l
  ON c.id = l.converted_contact;


In [0]:
%sql
SELECT 
  COUNT(DISTINCT c.id) AS common_unique_ids,
  COUNT(DISTINCT CASE 
    WHEN c.data_criacao_da_lead = l.created_time THEN c.id
  END) AS common_unique_ids_with_date
FROM workspace.sc_silver.contactos_pbs c
JOIN workspace.sc_silver.leads_pbs l
  ON c.id = l.converted_contact;


In [0]:
from pyspark.sql import functions as F

# ==== LEADS ====
df_leads = spark.table("workspace.sc_silver.leads_pbs")
df_leads_renamed = df_leads.select(
    *[F.col(c).alias(f"{c}_leads") for c in df_leads.columns]
)
df_leads_renamed.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.leads_renamed")
display(spark.table("workspace.sc_silver.leads_renamed"))

# ==== CONTACTOS ====
df_contactos = spark.table("workspace.sc_silver.contactos_pbs")
df_contactos_renamed = df_contactos.select(
    *[F.col(c).alias(f"{c}_contactos") for c in df_contactos.columns]
)
df_contactos_renamed.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.contactos_renamed")
display(spark.table("workspace.sc_silver.contactos_renamed"))


In [0]:
from pyspark.sql import functions as F

# Load both renamed tables
df_leads = spark.table("workspace.sc_silver.leads_renamed")
df_contactos = spark.table("workspace.sc_silver.contactos_renamed")

# LEFT join on both ID and date
df_joined = df_leads.join(
    df_contactos,
    (df_leads.converted_contact_leads == df_contactos.id_contactos) &
    (F.to_timestamp(df_leads.created_time_leads, "dd-MM-yyyy HH:mm") ==
     F.to_timestamp(df_contactos.data_criacao_da_lead_contactos, "dd-MM-yyyy HH:mm")),
    "left"
)

# Save as a new permanent table
df_joined.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.leads_with_contactos")

# Display preview
display(spark.table("workspace.sc_silver.leads_with_contactos"))


In [0]:
from pyspark.sql import functions as F

# Load the new table
df_new = spark.table("workspace.sc_silver.leads_with_contactos")

# Calculate the counts
df_counts = df_new.agg(
    F.countDistinct("id_contactos").alias("common_unique_ids"),
    F.countDistinct(
        F.when(
            F.to_timestamp(df_new.data_criacao_da_lead_contactos, "dd-MM-yyyy HH:mm") ==
            F.to_timestamp(df_new.created_time_leads, "dd-MM-yyyy HH:mm"),
            df_new.id_contactos
        )
    ).alias("common_unique_ids_with_date")
)

df_counts.show()


In [0]:
%sql

SELECT *
FROM workspace.sc_silver.propostas_with_deals_with_campaigns
LIMIT 100;


In [0]:
%sql
-- First table
SELECT *
FROM workspace.sc_silver.leads_with_contactos
LIMIT 100;


In [0]:
from pyspark.sql import functions as F

# Load both tables
df_leads_contactos = spark.table("workspace.sc_silver.leads_with_contactos")
df_propostas = spark.table("workspace.sc_silver.propostas_with_deals_with_campaigns")

# LEFT join on id_contactos vs id_contacto_propostas_realizadas
df_funil = df_leads_contactos.join(
    df_propostas,
    df_leads_contactos.id_contactos == df_propostas.id_contacto_propostas_realizadas,
    "left"
)

# Save as permanent table
df_funil.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.Funil")

# Preview
display(spark.table("workspace.sc_silver.Funil"))


In [0]:
%sql
SHOW TABLES IN workspace.sc_silver;

In [0]:
%sql
-- Drop specific tables
DROP TABLE IF EXISTS sc_silver.funil;

DROP TABLE IF EXISTS sc_silver.campaigns_renamed;
DROP TABLE IF EXISTS sc_silver.contactos_renamed;
DROP TABLE IF EXISTS sc_silver.deals_renamed;
DROP TABLE IF EXISTS sc_silver.leads_renamed;
DROP TABLE IF EXISTS sc_silver.propostas_realizadas_renamed;



DROP TABLE IF EXISTS sc_silver.deals_with_campaigns;
DROP TABLE IF EXISTS sc_silver.leads_with_contactos;
DROP TABLE IF EXISTS sc_silver.propostas_with_deals_with_campaigns;


In [0]:
%sql
DROP TABLE IF EXISTS sc_silver.campaigns_cleaned;
DROP TABLE IF EXISTS sc_silver.contactos_cleaned;
DROP TABLE IF EXISTS sc_silver.deals_cleaned;
DROP TABLE IF EXISTS sc_silver.leads_cleaned;
DROP TABLE IF EXISTS sc_silver.propostas_realizadas_cleaned;

In [0]:
%sql
DROP TABLE IF EXISTS sc_silver.bd_rede_hyundai_cleaned;
DROP TABLE IF EXISTS sc_silver.campanha_vouchers_cleaned;
DROP TABLE IF EXISTS sc_silver.campanhas_tecnicas_cleaned;
DROP TABLE IF EXISTS sc_silver.contactos_importador_pbs_cleaned;
DROP TABLE IF EXISTS sc_silver.contratos_financiamento_cleaned;
DROP TABLE IF EXISTS sc_silver.viaturas_cleaned;
DROP TABLE IF EXISTS sc_silver.viaturas_demo_cleaned;

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StringType, DoubleType, FloatType
)

# Load table
df = spark.table("sc_silver.funil")

total_rows = df.count()
if total_rows == 0:
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs = []
    col_names = []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    tidy.orderBy(F.desc("blank_pct")).show(truncate=False)


In [0]:
tidy.orderBy(F.desc("blank_pct")).show(n=tidy.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, FloatType

tables = [
    "bd_rede_hyundai",
    "campanha_vouchers",
    "campanhas_tecnicas",
    "contactos_importador_pbs",
    "contratos_financiamento",
    "historico_de_servicos",
    "viaturas",
    "viaturas_demo"
]

def blank_percentage_for_table(table_name):
    print(f"=== Processing table: {table_name} ===")
    df = spark.table(f"sc_silver.{table_name}")
    total_rows = df.count()

    if total_rows == 0:
        print(f"Table {table_name} is empty.")
        return spark.createDataFrame([], "column string, blank_pct double")

    agg_exprs = []
    col_names = []

    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    pct_row = df.agg(*agg_exprs)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    return tidy.orderBy(F.desc("blank_pct"))

# Loop through all tables and show results
for tbl in tables:
    result_df = blank_percentage_for_table(tbl)
    result_df.show(n=result_df.count(), truncate=False)


In [0]:
%sql
-- Step 1: Enable column mapping (only needs to be done once for this table)
ALTER TABLE sc_silver.campanha_vouchers
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.campanha_vouchers
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop the columns individually (with IF EXISTS to avoid errors)
ALTER TABLE sc_silver.campanha_vouchers DROP COLUMN IF EXISTS email;
ALTER TABLE sc_silver.campanha_vouchers DROP COLUMN IF EXISTS unsubscribed_time;
ALTER TABLE sc_silver.campanha_vouchers DROP COLUMN IF EXISTS tag;
ALTER TABLE sc_silver.campanha_vouchers DROP COLUMN IF EXISTS secondary_email;
ALTER TABLE sc_silver.campanha_vouchers DROP COLUMN IF EXISTS unsubscribed_mode;


In [0]:
%sql
-- Step 1: Enable column mapping (only once per table)
ALTER TABLE sc_silver.campanhas_tecnicas
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.campanhas_tecnicas
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (IF EXISTS avoids errors if missing)
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS email;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS unsubscribed_mode;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS tag;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS unsubscribed_time;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS secondary_email;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.contactos_importador_pbs
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.contactos_importador_pbs
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS city;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS chave_instalacao_bd_rede;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS instalacao;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS ticket_suporte;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS tag;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS region;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS country;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS pioneiros_myhyundai;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS chave_concessao_bd_rede;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS equipa;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS formulario;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.contratos_financiamento
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.contratos_financiamento
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS secondary_email;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS unsubscribed_mode;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS unsubscribed_time;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS valor_residual_da_viatura;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS tag;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.viaturas
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.viaturas
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS description_dtc_5;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS secondary_email;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS description_dtc_1;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS description_dtc_10;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS description_dtc_4;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS n__da_fatura;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS description_dtc_7;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS email;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS preco_unit_venda;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS description_dtc_6;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS n__dav;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS canal_de_venda;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS financeira_da_viatura;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS description_dtc_8;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS description_dtc_3;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS unsubscribed_time;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS dealer_name;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS description_dtc_9;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS dealer_code;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS origem_viatura;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS cod_postal_concessao;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS description_dtc_2;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS endereco_concessao;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS unsubscribed_mode;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS tem_contrato_de_manutencao;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS tag;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS extensao_de_garantia;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.viaturas_demo
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.viaturas_demo
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS preco_base;
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS email;
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS pintura;
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS secondary_email;
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS unsubscribed_mode;
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS iva;
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS isv;
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS unsubscribed_time;
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS motivo;
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS sgpu;
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS tag;


In [0]:
%sql
SELECT * FROM sc_silver.bd_rede_hyundai;

In [0]:
%sql
SELECT * FROM sc_silver.campanha_vouchers;

In [0]:
%sql
SELECT * FROM sc_silver.campanhas_tecnicas;

In [0]:
%sql
SELECT * FROM sc_silver.contactos_importador_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contactos_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contas_importador_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contas_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contratos_financiamento;

In [0]:
%sql
SELECT * FROM sc_silver.deals;

In [0]:
%sql
SELECT * FROM sc_silver.historico_de_servicos;

In [0]:
%sql
SELECT * FROM sc_silver.leads_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.propostas_realizadas;

In [0]:
%sql
SELECT * FROM sc_silver.viaturas;

In [0]:
%sql
SELECT * FROM sc_silver.viaturas_demo;

In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.bd_rede_hyundai
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.bd_rede_hyundai
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS row_names;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS aftersales_performance_grouping_dt_changed;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS modified_time;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS last_activity_time;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS sales_area_manager_dt_changed;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS aftersales_area_manager_code;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS aftersales_area_manager_dt_changed;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS sales_team_code;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS aftersales_team_code;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS sales_team_dt_changed;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS billing_client_nav_code;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS aftersales_team_dt_changed;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS billing_client_nav_value;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS nif_grouping_code;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS nif_grouping_value;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS nif_grouping_dt_changed;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS nif;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS billing_client_nav_dt_changed;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS dealer_contract_code;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS sales_performance_grouping_code;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS dealer_contract_dt_changed;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS aftersales_performance_grouping_code;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS sales_performance_grouping_dt_changed;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS cp3;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS cp4;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS client_group_code;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS client_group_dt_changed;
ALTER TABLE sc_silver.bd_rede_hyundai DROP COLUMN IF EXISTS _fivetran_index;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.campanha_vouchers
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.campanha_vouchers
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.campanha_vouchers DROP COLUMN IF EXISTS row_names;
ALTER TABLE sc_silver.campanha_vouchers DROP COLUMN IF EXISTS email_opt_out;
ALTER TABLE sc_silver.campanha_vouchers DROP COLUMN IF EXISTS last_activity_time;
ALTER TABLE sc_silver.campanha_vouchers DROP COLUMN IF EXISTS modified_time;
ALTER TABLE sc_silver.campanha_vouchers DROP COLUMN IF EXISTS _fivetran_index;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.campanhas_tecnicas
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.campanhas_tecnicas
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS last_activity_time;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS modified_time;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS row_names;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS _fivetran_index;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS campanha_tecnica_owner_name;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS atualizarlink;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS ckd_plant;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS email_opt_out;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS visiting_owner;
ALTER TABLE sc_silver.campanhas_tecnicas DROP COLUMN IF EXISTS odometer_type;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.contactos_importador_pbs
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.contactos_importador_pbs
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS row_names;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS apelido;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS contacto_importador_owner_name;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS email;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS secondary_email;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS last_activity_time;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS modified_time;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS telemovel;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS telefone;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS nif;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS morada;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS codigo_postal;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS localidade;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS login_myhyundai;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS cctemp;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS atualizacao_cc_manual;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS nao_atualizar_concessao;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS ultimo_login;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS unsubscribed_mode;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS unsubscribed_time;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS domain;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS domain_age_days;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS enviar_sms;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS _fivetran_index;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS sub_status;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS smptp_provider;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS rgpd_art_17;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS mx_record;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS tipo_de_pedido;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS genero;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS age;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.contactos_pbs
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.contactos_pbs
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.contactos_pbs DROP COLUMN IF EXISTS last_activity_time;
ALTER TABLE sc_silver.contactos_pbs DROP COLUMN IF EXISTS modified_time;
ALTER TABLE sc_silver.contactos_pbs DROP COLUMN IF EXISTS data_de_validade_de_consentimento_hyundai;
ALTER TABLE sc_silver.contactos_pbs DROP COLUMN IF EXISTS _fivetran_index;
ALTER TABLE sc_silver.contactos_pbs DROP COLUMN IF EXISTS corporate;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.contas_importador_pbs
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.contas_importador_pbs
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS row_names;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS conta_importador_name;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS conta_importador_owner_name;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS modified_time;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS last_activity_time;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS tag;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS motivo;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS localidade;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS morada;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS concelho;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS codigo_postal;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS formulario;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS nipc;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS e_mail_da_empresa;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS telephone_da_empresa;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS _fivetran_index;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.contas_pbs
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.contas_pbs
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS _fivetran_index;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS row_names;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS conta_name;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS nipc;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS tag;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS e_mail_da_empresa;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS telefone_da_empresa;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS morada;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS corporate;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.contratos_financiamento
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.contratos_financiamento
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS row_names;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS modified_time;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS email_opt_out;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS last_activity_time;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS contratos_financiamento_owner_name;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS n_;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS auto_lead;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS vat_reclaimable;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS _fivetran_index;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.deals
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.deals
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.deals DROP COLUMN IF EXISTS data_de_matricula;
ALTER TABLE sc_silver.deals DROP COLUMN IF EXISTS task_created;
ALTER TABLE sc_silver.deals DROP COLUMN IF EXISTS corporate;
ALTER TABLE sc_silver.deals DROP COLUMN IF EXISTS _fivetran_index;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.historico_de_servicos
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.historico_de_servicos
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.historico_de_servicos DROP COLUMN IF EXISTS row_names;
ALTER TABLE sc_silver.historico_de_servicos DROP COLUMN IF EXISTS servico_pos_venda_owner_name;
ALTER TABLE sc_silver.historico_de_servicos DROP COLUMN IF EXISTS modified_time;
ALTER TABLE sc_silver.historico_de_servicos DROP COLUMN IF EXISTS last_activity_time;
ALTER TABLE sc_silver.historico_de_servicos DROP COLUMN IF EXISTS detalhe_spv;
ALTER TABLE sc_silver.historico_de_servicos DROP COLUMN IF EXISTS email_opt_out;
ALTER TABLE sc_silver.historico_de_servicos DROP COLUMN IF EXISTS spv_duplicado;
ALTER TABLE sc_silver.historico_de_servicos DROP COLUMN IF EXISTS _fivetran_index;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.leads_pbs
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.leads_pbs
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.leads_pbs DROP COLUMN IF EXISTS modified_time;
ALTER TABLE sc_silver.leads_pbs DROP COLUMN IF EXISTS last_activity_time;
ALTER TABLE sc_silver.leads_pbs DROP COLUMN IF EXISTS email_opt_out;
ALTER TABLE sc_silver.leads_pbs DROP COLUMN IF EXISTS tipo_de_lead;
ALTER TABLE sc_silver.leads_pbs DROP COLUMN IF EXISTS _fivetran_index;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.propostas_realizadas
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.propostas_realizadas
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.propostas_realizadas DROP COLUMN IF EXISTS sub_total;
ALTER TABLE sc_silver.propostas_realizadas DROP COLUMN IF EXISTS grand_total;
ALTER TABLE sc_silver.propostas_realizadas DROP COLUMN IF EXISTS _fivetran_index;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.viaturas
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.viaturas
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS row_names;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS modified_time;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS last_activity_time;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS extensao_promocional;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS check_test;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS _fivetran_index;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS gwms_verificado;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS wltp_co2;


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.viaturas_demo
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE sc_silver.viaturas_demo
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS row_names;
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS email_opt_out;
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS _fivetran_index;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StringType, DoubleType, FloatType
)

# Load table
df = spark.table("sc_silver.bd_rede_hyundai")

total_rows = df.count()
if total_rows == 0:
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs = []
    col_names = []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    tidy.orderBy(F.desc("blank_pct")).show(truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StringType, DoubleType, FloatType
)

# Load table
df = spark.table("sc_silver.campaigns")

total_rows = df.count()
if total_rows == 0:
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs = []
    col_names = []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    tidy.orderBy(F.desc("blank_pct")).show(truncate=False)


In [0]:
%sql
-- Step 1: Enable column mapping (only once for this table)
ALTER TABLE sc_silver.campaigns
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure minimum protocol version
ALTER TABLE  sc_silver.campaigns
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Step 2: Drop columns individually (use IF EXISTS to avoid errors if missing)

ALTER TABLE  sc_silver.campaigns DROP COLUMN IF EXISTS _fivetran_index;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StringType, DoubleType, FloatType
)

# Load table
df = spark.table("sc_silver.campanha_vouchers")

total_rows = df.count()
if total_rows == 0:
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs = []
    col_names = []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    tidy.orderBy(F.desc("blank_pct")).show(truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StringType, DoubleType, FloatType
)

# Load table
df = spark.table("sc_silver.campanhas_tecnicas")

total_rows = df.count()
if total_rows == 0:
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs = []
    col_names = []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    tidy.orderBy(F.desc("blank_pct")).show(truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, FloatType

# ---------- Config ----------
table_name = "sc_silver.contactos_importador_pbs"  # <- change if needed
only_above_pct = None  # e.g. set to 0 or 50 to filter; use None to show ALL
# ----------------------------

df = spark.table(table_name)

total_rows = df.count()
if total_rows == 0:
    print(f"{table_name} is empty.")
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs, col_names = [], []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    # Optional filter
    if only_above_pct is not None:
        tidy = tidy.filter(F.col("blank_pct") > float(only_above_pct))

    tidy = tidy.orderBy(F.desc("blank_pct"))

    # >>> Show ALL rows
    tidy.show(tidy.count(), truncate=False)


In [0]:
%sql
-- Step 1: Enable column mapping if not already set
ALTER TABLE sc_silver.contactos_importador_pbs
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure protocol version supports DROP COLUMN
ALTER TABLE sc_silver.contactos_importador_pbs
  SET TBLPROPERTIES ('delta.minReaderVersion'='2', 'delta.minWriterVersion'='5');

-- Step 2: Drop the specified columns individually (IF EXISTS avoids errors)
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS concelho;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS gender;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS firstname;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS lastname;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS data_de_nascimento;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS distrito;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS ativacao_conta_myhyundai;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS origem_registo;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS sexo;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS data_de_validade_de_consentimento_hyundai;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS link_centro_consentimento;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS consentimento;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS data_de_recolha_de_consentimento_hyundai;


In [0]:
spark.table("sc_silver.contactos_importador_pbs") \
    .select("pais") \
    .distinct() \
    .show(truncate=False)


In [0]:
spark.table("sc_silver.contactos_importador_pbs") \
    .groupBy("pais") \
    .count() \
    .orderBy(F.desc("count")) \
    .show(truncate=False)


In [0]:
%sql
-- Step 1: Enable column mapping if not already set
ALTER TABLE sc_silver.contactos_importador_pbs
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure protocol version supports DROP COLUMN
ALTER TABLE sc_silver.contactos_importador_pbs
  SET TBLPROPERTIES ('delta.minReaderVersion'='2', 'delta.minWriterVersion'='5');

-- Step 2: Drop the column (IF EXISTS avoids errors if already removed)
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS pais;


In [0]:
%sql
-- Step 1: Enable column mapping if not already set
ALTER TABLE sc_silver.contactos_importador_pbs
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Optional: ensure protocol version supports DROP COLUMN
ALTER TABLE sc_silver.contactos_importador_pbs
  SET TBLPROPERTIES ('delta.minReaderVersion'='2', 'delta.minWriterVersion'='5');

-- Step 2: Drop the specified columns (use IF EXISTS to avoid errors)
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS email_opt_out;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS check;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS free_email;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS mx_found;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS mx_founds;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS rgdp_art_17;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS smtp_provider;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS status;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS estado_email;
ALTER TABLE sc_silver.contactos_importador_pbs DROP COLUMN IF EXISTS account;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, FloatType

# ---------- Config ----------
table_name = "sc_silver.contactos_pbs"  # <- changed to contactos_pbs
only_above_pct = None  # e.g. set to 0 or 50 to filter; use None to show ALL
# ----------------------------

df = spark.table(table_name)

total_rows = df.count()
if total_rows == 0:
    print(f"{table_name} is empty.")
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs, col_names = [], []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    # Optional filter
    if only_above_pct is not None:
        tidy = tidy.filter(F.col("blank_pct") > float(only_above_pct))

    tidy = tidy.orderBy(F.desc("blank_pct"))

    # >>> Show ALL rows
    tidy.show(tidy.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, FloatType

# ---------- Config ----------
table_name = "sc_silver.contas_importador_pbs"  # <- changed to contas_importador_pbs
only_above_pct = None  # e.g. set to 0 or 50 to filter; use None to show ALL
# ----------------------------

df = spark.table(table_name)

total_rows = df.count()
if total_rows == 0:
    print(f"{table_name} is empty.")
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs, col_names = [], []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    # Optional filter
    if only_above_pct is not None:
        tidy = tidy.filter(F.col("blank_pct") > float(only_above_pct))

    tidy = tidy.orderBy(F.desc("blank_pct"))

    # >>> Show ALL rows
    tidy.show(tidy.count(), truncate=False)


In [0]:
%sql
-- Enable column mapping (needed for DROP COLUMN)
ALTER TABLE sc_silver.contas_importador_pbs
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Ensure protocol supports column drops
ALTER TABLE sc_silver.contas_importador_pbs
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Drop requested columns (safe with IF EXISTS)
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS distrito;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS origem_registo;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS pais;
ALTER TABLE sc_silver.contas_importador_pbs DROP COLUMN IF EXISTS telefone_da_empresa;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, FloatType

# ---------- Config ----------
table_name = "sc_silver.contas_pbs"  # <- changed to contas_pbs
only_above_pct = None  # e.g. set to 0 or 50 to filter; use None to show ALL
# ----------------------------

df = spark.table(table_name)

total_rows = df.count()
if total_rows == 0:
    print(f"{table_name} is empty.")
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs, col_names = [], []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    # Optional filter
    if only_above_pct is not None:
        tidy = tidy.filter(F.col("blank_pct") > float(only_above_pct))

    tidy = tidy.orderBy(F.desc("blank_pct"))

    # >>> Show ALL rows
    tidy.show(tidy.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

table = "sc_silver.contas_pbs"
cols = ["tasks_involved", "events_involved", "calls_involved", "rent_a_car"]

df = spark.table(table)
total = df.count()

# Only keep columns that actually exist (avoid errors)
present = [c for c in cols if c in df.columns]
missing = [c for c in cols if c not in df.columns]
if missing:
    print("Skipping missing columns:", ", ".join(missing))

for c in present:
    print(f"\n=== {c} ===")
    # Treat NULLs explicitly so they show up in counts
    out = (
        df.groupBy(F.coalesce(F.col(c).cast("string"), F.lit("<NULL>")).alias(c))
          .count()
          .withColumn("pct", F.round(F.col("count") / F.lit(total) * 100, 2))
          .orderBy(F.desc("count"))
    )
    out.show(out.count(), truncate=False)  # show all rows for this column


In [0]:
%sql
-- Enable column mapping (needed for DROP COLUMN in Delta)
ALTER TABLE sc_silver.contas_pbs
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Ensure protocol supports column drops (harmless if already set)
ALTER TABLE sc_silver.contas_pbs
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Drop the requested columns (IF EXISTS avoids errors if any are missing)
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS website;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS industry;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS pioneiros_myhyundai;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS login_myhyundai;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS cliente_hyundai;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS conta_type;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS motivo;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS id_lead_conversao;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS data_criacao_da_lead;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS agrupamento_performance_vendas;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS chave_agrupamento_performance_vendas;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS modo_de_contacto_preferencial;
ALTER TABLE sc_silver.contas_pbs DROP COLUMN IF EXISTS tipo_empresa;


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.contratos_financiamento"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [
        F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols
    ]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
%sql
-- Enable column mapping (safe to run again)
ALTER TABLE sc_silver.contratos_financiamento
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Ensure protocol supports DROP COLUMN
ALTER TABLE sc_silver.contratos_financiamento
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Drop requested columns
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS contacto_vinculado_id;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS data_criacao_da_lead;
ALTER TABLE sc_silver.contratos_financiamento DROP COLUMN IF EXISTS data_criacao_lead_financial_swap;


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.deals"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.historico_de_servicos"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

table = "sc_silver.historico_de_servicos"
cols = ["ciclo_de_fidelizacao", "dadosconcessao"]

df = spark.table(table)
total = df.count()

# Only keep columns that actually exist (avoid errors)
present = [c for c in cols if c in df.columns]
missing = [c for c in cols if c not in df.columns]
if missing:
    print("Skipping missing columns:", ", ".join(missing))

for c in present:
    print(f"\n=== {c} ===")
    # Treat NULLs explicitly so they show up in counts
    out = (
        df.groupBy(F.coalesce(F.col(c).cast("string"), F.lit("<NULL>")).alias(c))
          .count()
          .withColumn("pct", F.round(F.col("count") / F.lit(total) * 100, 2))
          .orderBy(F.desc("count"))
    )
    out.show(out.count(), truncate=False)  # show all rows for this column


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.leads_pbs"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.propostas_realizadas"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.viaturas"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.viaturas"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.viaturas_demo"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # Compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
%sql
-- =======================
-- sc_silver.viaturas
-- =======================
-- Enable column mapping (needed for DROP COLUMN in Delta)
ALTER TABLE sc_silver.viaturas
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Ensure protocol supports column drops (harmless if already set)
ALTER TABLE sc_silver.viaturas
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Drop the requested columns (IF EXISTS avoids errors if any are missing)
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS `prov_status`;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS `model_year`;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS `engine_type`;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS `country`;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS `no`;
ALTER TABLE sc_silver.viaturas DROP COLUMN IF EXISTS `model`;


In [0]:
%sql
-- =======================
-- sc_silver.viaturas_demo
-- =======================
-- Enable column mapping (needed for DROP COLUMN in Delta)
ALTER TABLE sc_silver.viaturas_demo
  SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Ensure protocol supports column drops (harmless if already set)
ALTER TABLE sc_silver.viaturas_demo
  SET TBLPROPERTIES ('delta.minReaderVersion'='2','delta.minWriterVersion'='5');

-- Drop the requested column
ALTER TABLE sc_silver.viaturas_demo DROP COLUMN IF EXISTS `origem_registo`;


In [0]:
%sql
SHOW TABLES IN workspace.sc_silver;

In [0]:
%sql
SELECT * FROM sc_silver.contas_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contas_importador_pbs;


In [0]:
%sql
SELECT * FROM sc_silver.campanha_vouchers;

In [0]:
%sql
SELECT * FROM sc_silver.campaigns;

In [0]:
%sql
SELECT * FROM sc_silver.bd_rede_hyundai;

In [0]:
%sql
SELECT * FROM sc_silver.campanhas_tecnicas;

In [0]:
%sql
SELECT * FROM sc_silver.contactos_importador_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contactos_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contratos_financiamento;

In [0]:
%sql
SELECT * FROM sc_silver.deals;

In [0]:
%sql
SELECT * FROM sc_silver.historico_de_servicos;

In [0]:
%sql
SELECT * FROM sc_silver.leads_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.propostas_realizadas;

In [0]:
%sql
SELECT * FROM sc_silver.viaturas;

In [0]:
%sql

SELECT * FROM sc_silver.viaturas_demo;

In [0]:
%sql
-- 1) See each duplicate pair and how many times it appears
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_silver.leads_pbs
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC;

-- 2) Count total duplicate rows (excluding first occurrence in each group)
WITH grouped AS (
    SELECT id, created_time, COUNT(*) AS cnt
    FROM sc_silver.leads_pbs
    GROUP BY id, created_time
)
SELECT SUM(cnt - 1) AS total_duplicate_rows
FROM grouped
WHERE cnt > 1;


In [0]:
%sql
SELECT *
FROM (
    SELECT *,
           COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
    FROM sc_silver.leads_pbs
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
SELECT a.*
FROM sc_silver.leads_pbs a
JOIN (
  SELECT _fivetran_id, ROW_NUMBER() OVER (PARTITION BY id, created_time ORDER BY _fivetran_id) rn
  FROM sc_silver.leads_pbs
) b
ON a._fivetran_id = b._fivetran_id
WHERE b.rn > 1
ORDER BY a.id, a.created_time;


In [0]:
%sql
-- Keep the first row per (id, created_time) and delete the rest
WITH ranked AS (
  SELECT
      _fivetran_id,
      ROW_NUMBER() OVER (
        PARTITION BY id, created_time
        ORDER BY _fivetran_id      -- any stable tiebreaker works
      ) AS rn
  FROM sc_silver.leads_pbs
)
DELETE FROM sc_silver.leads_pbs
WHERE _fivetran_id IN (SELECT _fivetran_id FROM ranked WHERE rn > 1);


In [0]:
%sql
-- 1) See each duplicate pair and how many times it appears
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_silver.leads_pbs
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC;

-- 2) Count total duplicate rows (excluding first occurrence in each group)
WITH grouped AS (
    SELECT id, created_time, COUNT(*) AS cnt
    FROM sc_silver.leads_pbs
    GROUP BY id, created_time
)
SELECT SUM(cnt - 1) AS total_duplicate_rows
FROM grouped
WHERE cnt > 1;

In [0]:
%sql
SELECT *
FROM (
    SELECT *,
           COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
    FROM sc_silver.leads_pbs
) t
WHERE cnt > 1
ORDER BY id, created_time;

In [0]:
%sql
-- 1) Duplicate (id, data_criacao_da_lead) pairs and their counts
SELECT id, data_criacao_da_lead, COUNT(*) AS cnt
FROM sc_silver.Contactos_PBS
GROUP BY id, data_criacao_da_lead
HAVING COUNT(*) > 1
ORDER BY cnt DESC;

-- 2) Total number of duplicate rows (excluding the first in each pair)
WITH grouped AS (
  SELECT id, data_criacao_da_lead, COUNT(*) AS cnt
  FROM sc_silver.Contactos_PBS
  GROUP BY id, data_criacao_da_lead
)
SELECT COALESCE(SUM(cnt - 1), 0) AS total_duplicate_rows
FROM grouped
WHERE cnt > 1;


In [0]:
%sql
SELECT *
FROM (
  SELECT *,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contactos_pbs
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- Keep the first row per (id, created_time) and delete the rest
WITH ranked AS (
  SELECT
      _fivetran_id,
      ROW_NUMBER() OVER (
        PARTITION BY id, created_time
        ORDER BY _fivetran_id          -- change this if you prefer a different keeper
      ) AS rn
  FROM sc_silver.contactos_pbs
)
DELETE FROM sc_silver.contactos_pbs
WHERE _fivetran_id IN (SELECT _fivetran_id FROM ranked WHERE rn > 1);


In [0]:
%sql
-- 1) Duplicate (id, created_time) pairs and their counts
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_silver.contactos_pbs
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id, created_time;

-- 2) Total duplicate rows (excluding the first in each pair)
WITH grouped AS (
  SELECT id, created_time, COUNT(*) AS cnt
  FROM sc_silver.contactos_pbs
  GROUP BY id, created_time
)
SELECT COALESCE(SUM(cnt - 1), 0) AS total_duplicate_rows
FROM grouped
WHERE cnt > 1;


In [0]:
%sql
SELECT
  id,
  created_time,
  COUNT(*) AS cnt,
  COLLECT_SET(id_proposta_realizada) AS proposta_ids  -- use COLLECT_LIST if you want repeats
FROM sc_silver.propostas_realizadas
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id, created_time;


In [0]:
%sql
SELECT
  id,
  created_time,
  id_proposta_realizada,
  subject,
  proposta_realizada_owner,
  proposta_realizada_owner_name,
  cnt
FROM (
  SELECT *,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.propostas_realizadas
) t
WHERE cnt > 1
ORDER BY id, created_time, id_proposta_realizada;


In [0]:
%sql
WITH ranked AS (
  SELECT
      _fivetran_id,
      ROW_NUMBER() OVER (
        PARTITION BY id, created_time, id_proposta_realizada
        ORDER BY _fivetran_id
      ) AS rn
  FROM sc_silver.propostas_realizadas
)
DELETE FROM sc_silver.propostas_realizadas
WHERE _fivetran_id IN (SELECT _fivetran_id FROM ranked WHERE rn > 1);


In [0]:
%sql
SELECT *
FROM (
  SELECT *,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contactos_pbs
) t
WHERE cnt > 1
ORDER BY id, created_time;

In [0]:
%sql
-- Show the actual rows that belong to duplicated (id, created_time) pairs
SELECT *
FROM (
  SELECT d.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.deals d
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_silver.deals
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id, created_time;


In [0]:
%sql
-- (Optional) Preview rows that would be deleted
SELECT a.*
FROM sc_silver.deals a
JOIN (
  SELECT _fivetran_id,
         ROW_NUMBER() OVER (
           PARTITION BY id, created_time
           ORDER BY _fivetran_id          -- pick your preferred tiebreaker
         ) AS rn
  FROM sc_silver.deals
) r
ON a._fivetran_id = r._fivetran_id
WHERE r.rn > 1
ORDER BY a.id, a.created_time, a._fivetran_id;


In [0]:
%sql
-- Delete all but one row per (id, created_time)
WITH ranked AS (
  SELECT
      _fivetran_id,
      ROW_NUMBER() OVER (
        PARTITION BY id, created_time
        ORDER BY _fivetran_id          -- change if you want a different keeper
      ) AS rn
  FROM sc_silver.deals
)
DELETE FROM sc_silver.deals
WHERE _fivetran_id IN (SELECT _fivetran_id FROM ranked WHERE rn > 1);


In [0]:
%sql
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_silver.deals
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id, created_time;

In [0]:
%sql
-- sc_silver.contas_pbs
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contas_pbs t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- Delete all but one row per (id, created_time)
WITH ranked AS (
  SELECT
    _fivetran_id,
    ROW_NUMBER() OVER (
      PARTITION BY id, created_time
      ORDER BY _fivetran_id          -- e.g., use updated_at DESC to keep newest
    ) AS rn
  FROM sc_silver.contas_pbs
)
DELETE FROM sc_silver.contas_pbs
WHERE _fivetran_id IN (
  SELECT _fivetran_id FROM ranked WHERE rn > 1
);


In [0]:
%sql
-- sc_silver.contas_importador_pbs
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contas_importador_pbs t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- Delete all but one row per (id, created_time)
WITH ranked AS (
  SELECT
    _fivetran_id,
    ROW_NUMBER() OVER (
      PARTITION BY id, created_time
      ORDER BY _fivetran_id            -- e.g., use updated_at DESC to keep newest
    ) AS rn
  FROM sc_silver.contas_importador_pbs
)
DELETE FROM sc_silver.contas_importador_pbs
WHERE _fivetran_id IN (SELECT _fivetran_id FROM ranked WHERE rn > 1);


In [0]:
%sql
-- sc_silver.campanha_vouchers
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.campanha_vouchers t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.bd_rede_hyundai
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.bd_rede_hyundai t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.campanhas_tecnicas
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.campanhas_tecnicas t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.contactos_importador_pbs
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contactos_importador_pbs t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- Delete all but one row per (id, created_time)
WITH ranked AS (
  SELECT
    _fivetran_id,
    ROW_NUMBER() OVER (
      PARTITION BY id, created_time
      ORDER BY _fivetran_id          -- e.g., use updated_at DESC to keep newest
    ) AS rn
  FROM sc_silver.contactos_importador_pbs
)
DELETE FROM sc_silver.contactos_importador_pbs
WHERE _fivetran_id IN (SELECT _fivetran_id FROM ranked WHERE rn > 1);


In [0]:
%sql
-- sc_silver.contratos_financiamento
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contratos_financiamento t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.historico_de_servicos
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.historico_de_servicos t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.viaturas
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.viaturas t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.viaturas_demo
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.viaturas_demo t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
# Databricks: run as a Python cell
from pyspark.sql import functions as F, types as T

TABLES = [
    "sc_silver.bd_rede_hyundai",
    "sc_silver.campaigns",
    "sc_silver.campanha_vouchers",
    "sc_silver.campanhas_tecnicas",
    "sc_silver.contactos_importador_pbs",
    "sc_silver.contactos_pbs",
    "sc_silver.contas_importador_pbs",
    "sc_silver.contas_pbs",
    "sc_silver.contratos_financiamento",
    "sc_silver.deals",
    "sc_silver.historico_de_servicos",
    "sc_silver.leads_pbs",
    "sc_silver.propostas_realizadas",
    "sc_silver.viaturas",
    "sc_silver.viaturas_demo",
]

def preserve_case(colname: str) -> bool:
    """
    Columns to NOT proper-case (keep original case, only trim):
    IDs, emails, URLs, VIN/IBAN, geo codes, date/time-like, etc.
    """
    c = colname.lower()
    if c == "id" or c.endswith("_id") or c.endswith("id"):
        return True
    sensitive_terms = [
        "email", "mail", "url", "http", "https",
        "vin", "iban", "bic", "swift", "uuid", "guid", "imei", "ip", "mac",
        "nif", "nipc", "niss", "vat",
        "postal", "codigo_postal", "zip", "cep",
        "matricula", "placa", "chassis", "plate",
        "lat", "lng", "lon", "latitude", "longitude",
    ]
    if any(term in c for term in sensitive_terms):
        return True
    # date/time-like names
    if "date" in c or "data" in c or "time" in c or "timestamp" in c:
        return True
    return False

def clean_string_col(name: str):
    col = F.col(name)
    # Trim ONLY ends; DO NOT normalize internal spaces or remove control chars; DO NOT map "" -> NULL
    col = F.trim(col)
    # Proper-case (per word) unless column should preserve case
    if not preserve_case(name):
        col = F.initcap(F.lower(col))  # e.g., "JOÃO DA SILVA" -> "João Da Silva"
    return col.alias(name)

def clean_df(df):
    out = []
    for f in df.schema.fields:
        if isinstance(f.dataType, T.StringType):
            out.append(clean_string_col(f.name))
        else:
            # non-string columns unchanged
            out.append(F.col(f.name))
    return df.select(*out)

for full_name in TABLES:
    df = spark.table(full_name)
    df_clean = clean_df(df)
    # Overwrite original table in place (Delta)
    df_clean.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(full_name)
    print(f"Overwrote: {full_name}")
